# Importacion de librerias

In [9]:
import pandas as pd
import boto3
from dotenv import load_dotenv
import os

# Descarga de los datos desde un compomente de S3 en AWS

In [10]:

load_dotenv('../.env')
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')

# Parámetros del bucket
bucket_name = 'assessment-86fc5eb8'
key = 'raw-data/data.csv'
local_path = '../data/data.csv'

# Crear carpeta si no existe
os.makedirs(os.path.dirname(local_path), exist_ok=True)


# Sesión con credenciales explícitas
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name='us-east-1' 
)

s3 = session.client('s3')

# Descargar archivo
s3.download_file(bucket_name, key, local_path)

# Importacion de los datos

In [11]:
df = pd.read_csv(local_path)
df.head(10)

,fecha_compra,usuario,producto,cantidad,precio
0,2014-10-01,6cba0c,PROD-efd583,2,164.7
1,2014-10-01,6cba0c,PROD-a726f5,2,164.7
2,2014-10-01,6cba0c,PROD-3e1cd9,2,164.7
3,2014-10-01,6cba0c,PROD-d012c0,2,164.7
4,2014-10-01,6cba0c,PROD-48924a,3,164.7
5,2014-10-01,6cba0c,PROD-4c2559,2,164.7
6,2014-10-01,6cba0c,PROD-10e913,2,164.7
7,2014-10-01,6cba0c,PROD-3065f9,2,164.7
8,2014-10-01,6cba0c,PROD-f47365,2,164.7
9,2014-10-01,6cba0c,PROD-e658d1,1,164.7


In [12]:
num_usuarios=len(df['usuario'].unique())
num_productos=len(df['producto'].unique())
print(f"Número de usuarios únicos : {num_usuarios} y número de productos únicos: {num_productos}")

Número de usuarios únicos : 197392 y número de productos únicos: 16854


# Paso 1: Identificar ¿cuales son los clientes recurrentes

#### Los clientes recurrentes deben cumplir esta condicion. Un cliente es recurrente si:

Hace más de una compra.
(Es decir, el cliente ha comprado al menos dos veces.)

Cada compra tiene más de 10 productos.
(En cada compra que cuenta, la cantidad total de productos debe ser mayor a 10.)

Las compras ocurren dentro de períodos de 30 días.
(Entre una compra y la siguiente, pasan 30 días o menos.)

In [13]:
# Convertir fecha a datetime
df['fecha_compra'] = pd.to_datetime(df['fecha_compra'])

# Agrupar por usuario y fecha, sumar productos comprados
compras = df.groupby(['usuario', 'fecha_compra']).agg(
    productos_comprados=('cantidad', 'sum')
).reset_index()

# Filtrar compras con más de 10 productos
compras_validas = compras[compras['productos_comprados'] > 10].copy()

# Ordenar por usuario y fecha
compras_validas = compras_validas.sort_values(['usuario', 'fecha_compra'])
compras_validas.head(10)

,usuario,fecha_compra,productos_comprados
0,000019,2014-10-26,42
1,000019,2014-12-24,25
3,00009c,2014-12-16,47
8,000166,2014-11-08,12
9,000166,2014-11-25,37
11,000342,2014-10-31,14
12,000342,2014-12-06,16
15,000481,2014-11-15,113
16,00065b,2014-10-29,157
18,0006ec,2014-12-28,82


Identificar los clientes que hacen más de una compra

In [14]:
# Contar número de compras válidas por usuario
conteo_compras = compras_validas.groupby('usuario').size().reset_index(name='num_compras_validas')

# Filtrar solo usuarios con más de una compra válida
usuarios_mas_de_una_compra = conteo_compras[conteo_compras['num_compras_validas'] > 1]

# Unir esa información a la tabla de compras válidas
compras_recurrentes = compras_validas.merge(usuarios_mas_de_una_compra, on='usuario')

# Resultado
print(compras_recurrentes.head())

  usuario fecha_compra  productos_comprados  num_compras_validas
0  000019   2014-10-26                   42                    2
1  000019   2014-12-24                   25                    2
2  000166   2014-11-08                   12                    2
3  000166   2014-11-25                   37                    2
4  000342   2014-10-31                   14                    2


Por información ¿Cuántos de estos clientes me realizan más de dos compras? 

In [15]:
# Contar número de compras válidas por usuario
conteo_compras = compras_validas.groupby('usuario').size().reset_index(name='num_compras_validas')

# Filtrar usuarios con más de 2 compras válidas
usuarios_mas_de_dos_compras = conteo_compras[conteo_compras['num_compras_validas'] > 2]

# Mostrar cuántos son
print(f"Clientes con más de 2 compras válidas: {len(usuarios_mas_de_dos_compras)}")

Clientes con más de 2 compras válidas: 52814


Identificar de los clientes cuáles compran en un intervalo de 30 dias 

In [16]:
# 1. Calcular dif_dias
compras_validas['dif_dias'] = compras_validas.groupby('usuario')['fecha_compra'].diff().dt.days

# 2. Filtrar usuarios con más de una compra
multi_compra = compras_validas.groupby('usuario').filter(lambda g: len(g) > 1)

# 3. Filtrar compras donde alguna diferencia de días es <= 30
mask = (
    multi_compra
    .groupby('usuario')['dif_dias']
    .transform(lambda s: s.dropna().le(30).any())
)
compras_recurrentes_30d = multi_compra[mask].copy()

# 4. Usuarios únicos resultantes
usuarios_recurrentes = compras_recurrentes_30d['usuario'].unique()

print(f"Clientes recurrentes (compras en ≤ 30 días): {len(usuarios_recurrentes)}")
print(compras_recurrentes_30d.head())

Clientes recurrentes (compras en ≤ 30 días): 65448
   usuario fecha_compra  productos_comprados  dif_dias
8   000166   2014-11-08                   12       NaN
9   000166   2014-11-25                   37      17.0
40  000998   2014-12-25                   31       NaN
41  000998   2014-12-28                   56       3.0
44  000a09   2014-10-07                   42       NaN


Revisar las comprar de un cliente

In [17]:
cliente = '000166'
info_000166 = compras_recurrentes_30d[
    compras_recurrentes_30d['usuario'] == cliente
]

# Mostrar columnas clave del usuario que estoy revisando
print(info_000166[['usuario', 'fecha_compra', 'productos_comprados', 'dif_dias']])

  usuario fecha_compra  productos_comprados  dif_dias
8  000166   2014-11-08                   12       NaN
9  000166   2014-11-25                   37      17.0


# Objetivo: Construir una tabla RFM (Recency–Frequency–Monetary): 

#Priorizar acciones de marketing, Segmentar de forma objetiva, Diseñar promociones personalizadas

In [18]:
compras_recurrentes_30d = compras_recurrentes_30d.sort_values(['usuario', 'fecha_compra'])

# 2. Creamos funciones para calcular frecuencia y monto acumulados por usuario hasta cada fecha_compra
def calc_cumulative_counts(df):
    return df.groupby('usuario').cumcount() + 1  # frequency acumulado

def calc_cumulative_monetary(df):
    return df.groupby('usuario')['productos_comprados'].cumsum()  # monetary acumulado

compras_recurrentes_30d['frequency_dynamic'] = calc_cumulative_counts(compras_recurrentes_30d)
compras_recurrentes_30d['monetary_dynamic'] = calc_cumulative_monetary(compras_recurrentes_30d)

# 3. Para recency dinámico: para cada fila, recency es la diferencia en días entre la fecha_compra de esa fila y la última compra previa
# Primero calculamos la última fecha de compra anterior a la actual (shift por usuario)
compras_recurrentes_30d['last_purchase_prior'] = (
    compras_recurrentes_30d
    .groupby('usuario')['fecha_compra']
    .shift(1)
)

# Recency dinámico es la diferencia entre la fecha de la fila y la última compra previa
compras_recurrentes_30d['recency_dynamic'] = (
    (compras_recurrentes_30d['fecha_compra'] - compras_recurrentes_30d['last_purchase_prior'])
    .dt.days
)

# Nota: La primera compra por usuario tendrá recency_dynamic = NaN (porque no hay compra anterior)

# 4. Puedes llenar ese NaN con algún valor si quieres, por ejemplo 0 o un valor grande (depende de tu análisis)
compras_recurrentes_30d['recency_dynamic'] = compras_recurrentes_30d['recency_dynamic'].fillna(0)

# Ahora tienes variables RFM que cambian para cada fila y reflejan el historial acumulado hasta ese momento
print(compras_recurrentes_30d[['usuario', 'fecha_compra', 'frequency_dynamic', 'monetary_dynamic', 'recency_dynamic']].head(10))



   usuario fecha_compra  frequency_dynamic  monetary_dynamic  recency_dynamic
8   000166   2014-11-08                  1                12              0.0
9   000166   2014-11-25                  2                49             17.0
40  000998   2014-12-25                  1                31              0.0
41  000998   2014-12-28                  2                87              3.0
44  000a09   2014-10-07                  1                42              0.0
45  000a09   2014-11-27                  2                97             51.0
46  000a09   2014-12-04                  3               149              7.0
52  000b65   2014-10-08                  1                35              0.0
53  000b65   2014-10-16                  2                48              8.0
54  000b65   2014-10-25                  3                76              9.0


Incorporar la variable mes

In [19]:
# Convertir fecha_compra a datetime
compras_recurrentes_30d['fecha_compra'] = pd.to_datetime(compras_recurrentes_30d['fecha_compra'])

# Crear columna 'mes' en formato año-mes (ej: 2014-11)
compras_recurrentes_30d['mes'] = compras_recurrentes_30d['fecha_compra'].dt.month

# Verificar resultado
print(compras_recurrentes_30d[['usuario', 'fecha_compra', 'mes', 'frequency_dynamic', 'monetary_dynamic', 'recency_dynamic']].head(10))

# Creamos base_enriquecida como copia actualizada
base_enriquecida = compras_recurrentes_30d.copy()


   usuario fecha_compra  mes  frequency_dynamic  monetary_dynamic  \
8   000166   2014-11-08   11                  1                12   
9   000166   2014-11-25   11                  2                49   
40  000998   2014-12-25   12                  1                31   
41  000998   2014-12-28   12                  2                87   
44  000a09   2014-10-07   10                  1                42   
45  000a09   2014-11-27   11                  2                97   
46  000a09   2014-12-04   12                  3               149   
52  000b65   2014-10-08   10                  1                35   
53  000b65   2014-10-16   10                  2                48   
54  000b65   2014-10-25   10                  3                76   

    recency_dynamic  
8               0.0  
9              17.0  
40              0.0  
41              3.0  
44              0.0  
45             51.0  
46              7.0  
52              0.0  
53              8.0  
54              9.0 

# Predecir los 3 productos más probables (baseline sencillo)

In [20]:
import pandas as pd

# Asumo que df ya está cargado con columnas: usuario, producto, cantidad

# Agrupar productos por usuario y sumar las cantidades
productos_por_usuario = (
    df.groupby(['usuario', 'producto'])['cantidad']
    .sum()
    .reset_index()
)

# Ordenar productos por usuario según cantidad comprada descendente
productos_por_usuario = productos_por_usuario.sort_values(['usuario', 'cantidad'], ascending=[True, False])

# Crear lista con los productos ordenados por cantidad para cada usuario
productos_ordenados = (
    productos_por_usuario.groupby('usuario')['producto']
    .apply(list)
    .reset_index(name='productos_ordenados')
)


In [21]:
# Tomar solo los 3 primeros productos por usuario
productos_ordenados['top_3_productos'] = productos_ordenados['productos_ordenados'].apply(lambda x: x[:3])

In [22]:
# Suponiendo base_enriquecida tiene la columna usuario
base_con_productos = base_enriquecida.merge(
    productos_ordenados[['usuario', 'top_3_productos']],
    on='usuario',
    how='left'
)

# Ahora base_con_productos tiene la columna 'top_3_productos' con la predicción sencilla para cada usuario


In [23]:
print(base_con_productos[['usuario', 'top_3_productos']].head())

  usuario                          top_3_productos
0  000166  [PROD-62e82f, PROD-0256a8, PROD-4d5e04]
1  000166  [PROD-62e82f, PROD-0256a8, PROD-4d5e04]
2  000998  [PROD-74efc1, PROD-dab6fc, PROD-5eb939]
3  000998  [PROD-74efc1, PROD-dab6fc, PROD-5eb939]
4  000a09  [PROD-3bea74, PROD-412320, PROD-66a332]
